### NORTHWIND DATABASE ANALYSIS USING JUPYTER  & SQL

The Northwind database is a sample database that was originally created by Microsoft to demonstrate the features of its products, particularly Microsoft Access. It is often used as a sample database in educational settings, tutorials, and examples for learning and practicing database concepts.
The Northwind database represents a fictional company and contains tables for customers, orders, products, employees, and other entities typically found in a business environment. It's a relational database, and its structure is designed to showcase various aspects of database design and querying.


#### Objectives

Using this classic database we shall learn how to use String functions, Aggregations, CASE, Subquery, Window functions, CTEs, Views, Temporary Tables, and finally some Data Analysis.

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql

In [2]:
# Establish a connection to a SQLite database file.
%sql sqlite:///Northwind.db

'Connected: @Northwind.db'

### String Functions



#### 1. Use CONCAT(), LOWER() to create a new column Email

In SQLite, the CONCAT function is not directly available. Instead, you can use the concatenation operator (||). Concatenate joins multiple strings together. Lower() & UPPER() converts string to lowercase and uppercase respectively.

In [3]:
%%sql

SELECT
    FirstName,
    LastName,
    LOWER(FirstName)||'.'|| LOWER(LastName) || '@northwind.com' AS Email
FROM Employees
LIMIT 5;


 * sqlite:///Northwind.db
Done.


FirstName,LastName,Email
Nancy,Davolio,nancy.davolio@northwind.com
Andrew,Fuller,andrew.fuller@northwind.com
Janet,Leverling,janet.leverling@northwind.com
Margaret,Peacock,margaret.peacock@northwind.com
Steven,Buchanan,steven.buchanan@northwind.com


#### 2. Assume we want to send our customers promotional text messages.

We shall create a new column from the Phone column in the Customers table where we remove characters like (, ), -, ., and spaces. You can achieve this by using the REPLACE function multiple times.

In [4]:
%%sql

SELECT
    Phone,
    REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(Phone, '(',''), ')', ''), '-', ''), '.', ''), ' ', '') AS New_number
FROM Customers
LIMIT 10;


 * sqlite:///Northwind.db
Done.


Phone,New_number
030-0074321,0300074321
(5) 555-4729,55554729
(5) 555-3932,55553932
(171) 555-7788,1715557788
0921-12 34 65,0921123465
0621-08460,062108460
88.60.15.31,88601531
(91) 555 22 82,915552282
91.24.45.40,91244540
(604) 555-4729,6045554729


#### 3. Suppose we want to extract the first two characters from the QuantityPerUnit column , we use SUBSTR() function.

The below query extracts the first two characters of the QuantityPerUnit column and assigns it to a new column named 'Units@package'. 

In [5]:
%%sql
SELECT
      QuantityPerUnit,
      SUBSTR(QuantityPerUnit,1,2) as 'Units@package'
FROM Products
LIMIT 5;

 * sqlite:///Northwind.db
Done.


QuantityPerUnit,Units@package
10 boxes x 20 bags,10
24 - 12 oz bottles,24
12 - 550 ml bottles,12
48 - 6 oz jars,48
36 boxes,36


### Aggregations

#### 1. Total numbers of orders

In [6]:
%%sql

SELECT 
      COUNT(*) as Total_orders
    
FROM Orders

 * sqlite:///Northwind.db
Done.


Total_orders
830


#### 2. Customer with the most orders

In [7]:
%%sql

SELECT 
      CompanyName,
      COUNT(*) as Total_orders
        
FROM Orders O
JOIN Customers C
ON O.CustomerID = C.CustomerID
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

 * sqlite:///Northwind.db
Done.


CompanyName,Total_orders
Save-a-lot Markets,31
Ernst Handel,30
QUICK-Stop,28
Hungry Owl All-Night Grocers,19
Folk och f HB,19


#### 3. Average unit price of products in each category.

Query calculates the average unit price for each category and rounding the result to two decimal places. The JOIN clause combines the Products and Categories tables based on the CategoryID, and then the GROUP BY clause groups the results by CategoryName. The AVG function calculates the average unit price for each category, and the ROUND function ensures that the result is rounded to two decimal places. Finally, the ORDER BY clause orders the results in descending order based on the average unit price.

In [8]:
%%sql

SELECT
      C.CategoryName,
      ROUND(AVG(UnitPrice),2) as AVG_unit_price
        
FROM Products P
JOIN Categories C
ON P.CategoryID = C.CategoryID
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///Northwind.db
Done.


CategoryName,AVG_unit_price
Meat/Poultry,54.01
Beverages,37.98
Produce,32.37
Dairy Products,28.73
Confections,25.16
Condiments,23.06
Seafood,20.68
Grains/Cereals,20.25


#### 4. Customer with the highest total order amount.

The query calculates the total amount for each customer and orders the results by the total amount in descending order. The LIMIT 5 ensures that you only get the top 5 results.I used SUM to calculate the total amount for each customer, assuming that you want the total amount for all orders of each customer. Additionally, I wanted to remove the decimal part and obtain the integer value without rounding, I used the CAST function.

In [9]:
%%sql

SELECT 
      C.CompanyName,
      CAST(SUM(OD.UnitPrice * OD.Quantity) AS INT) AS Total_amount

FROM Customers C
JOIN Orders O
ON O.CustomerID = C.CustomerID
JOIN OrderDetails OD
ON O.OrderID = OD.OrderID
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

 * sqlite:///Northwind.db
Done.


CompanyName,Total_amount
QUICK-Stop,117483
Save-a-lot Markets,115673
Ernst Handel,113236
Hungry Owl All-Night Grocers,57317
Rattlesnake Canyon Grocery,52245


#### SUBQUERIES

######  Basic Subqueries

#### 1. Products with a price higher than the average price

The subquery (SELECT AVG(UnitPrice) FROM Products) calculates the average unit price for all products.
The main query selects the ProductName and UnitPrice for products where the UnitPrice is higher than the average price obtained from the subquery.

In [10]:
%%sql

SELECT
      AVG(UnitPrice) as Avg_unit_price
FROM Products;

 * sqlite:///Northwind.db
Done.


Avg_unit_price
28.866363636363637


In [11]:
%%sql

SELECT
      ProductName,
      UnitPrice
        
FROM Products
WHERE UnitPrice >  (SELECT
                           AVG(UnitPrice) 
                     FROM Products)
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///Northwind.db
Done.


ProductName,UnitPrice
Cte de Blaye,263.5
Thringer Rostbratwurst,123.79
Mishi Kobe Niku,97
Sir Rodney's Marmalade,81
Carnarvon Tigers,62.5
Raclette Courdavault,55
Manjimup Dried Apples,53
Tarte au sucre,49.3
Ipoh Coffee,46
Rssle Sauerkraut,45.6


#### 2.  Employees who have a salary greater than the average salary.

The query will give you a list of employees with their names and 
salaries where the salary is higher than the average salary, ordered by salary in descending order.

In [12]:
%%sql

SELECT
      LastName||' '||FirstName as Name,
      Salary
        
FROM Employees
WHERE Salary > (SELECT 
                      AVG(Salary)
                FROM Employees)
ORDER BY 2 DESC;

 * sqlite:///Northwind.db
Done.


Name,Salary
Leverling Janet,3119.15
Davolio Nancy,2954.55
Dodsworth Anne,2333.33


###### Correlated Subqueries:

A correlated subquery is a subquery that references one or more columns from the outer query. Unlike a non-correlated subquery, which is independent of the outer query, a correlated subquery depends on the values from the outer query.

We shall also use **Temporary tables**, a temporary table in SQL is a table that exists temporarily for the duration of a session or a transaction. It is typically used to store intermediate results that you need to reuse within the context of a specific session or transaction. They are useful when you need to store intermediate results, perform complex calculations, or break down a complex query into more manageable parts.

#### 1. Employees who have sales higher than the average sales of their respective country.

In [13]:
%%sql

-- Create a temporary table with the calculated total sales
CREATE TEMPORARY TABLE IF NOT EXISTS TempOrderDetails AS
SELECT
    E.EmployeeID,
    FirstName||' '||LastName as Name,
    E.Country,
    OD.UnitPrice * OD.Quantity AS TotalSales

FROM Employees E    
JOIN Orders O
ON E.EmployeeID = O.EmployeeID
JOIN OrderDetails OD 
ON O.OrderID = OD.OrderID;

-- Query using the temporary table with a correlated subquery
SELECT
      Name,
      T.Country,
      SUM(TotalSales)

FROM TempOrderDetails T
GROUP BY 1,2
HAVING SUM(TotalSales) > (SELECT 
                                average_sale
                          FROM (SELECT
                                      AVG(TotalSales) average_sale
                          FROM TempOrderDetails TI
                           WHERE TI.Country = T.Country));

 * sqlite:///Northwind.db
Done.
Done.


Name,Country,SUM(TotalSales)
Andrew Fuller,USA,177749.26000000004
Anne Dodsworth,UK,82963.99999999999
Janet Leverling,USA,213051.29999999996
Laura Callahan,USA,133301.02999999997
Margaret Peacock,USA,250187.44999999992
Michael Suyama,UK,78198.09999999999
Nancy Davolio,USA,202143.71000000002
Robert King,UK,141295.99
Steven Buchanan,UK,75567.75000000001


A temporary table named TempOrderDetails was created to store the calculated total sales for each employee and the employee Id, name and country.
The calculation of total sales is performed once and stored in the temporary table. Subsequent queries can refer to this table without repeating the same computations.
The use of the temporary table enhances code readability, avoids redundant computations, and facilitates the reuse of intermediate results in a structured manner. It's a helpful technique, especially when dealing with multi-step analyses or complex data transformations.

#### 3. Customers who have placed orders with a total amount less than the average order amount for their country.



In [14]:
%%sql

WITH Countries_total_orders AS
(SELECT 
      C.CompanyName,
      C.Country,
      SUM(OD.UnitPrice * OD.Quantity) AS Total_amount

FROM Customers C
JOIN Orders O
ON O.CustomerID = C.CustomerID
JOIN OrderDetails OD
ON O.OrderID = OD.OrderID
GROUP BY 1)

SELECT
      CompanyName,
      CTO.Country,
      ROUND(SUM(Total_amount)) as Total_order_amt
FROM Countries_total_orders CTO
GROUP BY 1
HAVING SUM(Total_amount) < (SELECT average_sales
                           FROM
                           (SELECT
                                  AVG(Total_amount) average_sales
                            FROM Countries_total_orders CTOI
                            WHERE CTO.Country = CTOI.Country))
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///Northwind.db
Done.


CompanyName,Country,Total_order_amt
GROSELLA-Restaurante,Venezuela,1489.0
Trail's Head Gourmet Provisioners,USA,1571.0
The Cracker Box,USA,1947.0
The Big Cheese,USA,3361.0
Split Rail Beer & Ale,USA,12490.0
Old World Delicatessen,USA,16325.0
Lonesome Pine Restaurant,USA,4259.0
Let's Stop N Shop,USA,3490.0
Lazy K Kountry Store,USA,357.0
Hungry Coyote Import Store,USA,3063.0


The query is using a **Common Table Expression (CTE)** named Countries_total_orders to calculate the total amount of orders for each customer's country. Then, the main query is using this CTE to filter and aggregate the results, specifically selecting companies whose total order amounts are less than the average total order amount for their respective countries.

The use of the WITH clause defines the CTE, and the main query then references and utilizes the results from the CTE.

The use of a correlated subquery is evident in the HAVING clause.
The subquery is correlated because it references the outer query's table CTO in the WHERE clause. Specifically, it's comparing the country of the current row in the outer query (CTO.Country) with the country of the rows in the inner query (CTOI.Country). This correlation establishes a relationship between the outer and inner queries.

The purpose of this correlated subquery is to calculate the average total amount of orders for the same country as the current row in the outer query. The HAVING clause then uses this correlated subquery to filter out the results, ensuring that only companies with total order amounts less than the average for their respective countries are included.

In summary, the correlated subquery helps conditionally filter the aggregated results in the HAVING clause based on the values of the current row in the outer query.

### CTE

A Common Table Expression (CTE) is a temporary result set that you can reference within a SELECT, INSERT, UPDATE, or DELETE statement. CTEs provide a way to define named temporary result sets in a SQL query, making complex queries more readable and manageable.

 #### Using a CTE, find the total sales for each category of products.

In [15]:
%%sql

WITH Product_category_sales AS 
(SELECT
      C.CategoryName,
      OD.UnitPrice,
      OD.Quantity,
      (OD.UnitPrice * OD.Quantity) as Sales
 
FROM orderDetails OD
JOIN Products P
ON P.ProductID = OD.ProductID
JOIN Categories C
ON C.CategoryID = P.CategoryID)

SELECT
      CategoryName,
      ROUND(SUM(Sales),2) as Total_sales

FROM Product_category_sales
GROUP BY 1
ORDER BY 2 DESC;


 * sqlite:///Northwind.db
Done.


CategoryName,Total_sales
Beverages,286526.95
Dairy Products,251330.5
Meat/Poultry,178188.8
Confections,177099.1
Seafood,141623.09
Condiments,113694.75
Produce,105268.6
Grains/Cereals,100726.8


#### CTE to show the monthly sales growth over time

In [16]:
%%sql

WITH Salestotals_overtime AS 
(    
SELECT
    O.OrderDate,
    C.CategoryName,
    OD.UnitPrice,
    OD.Quantity,
    (OD.UnitPrice * OD.Quantity) as Sales
    
FROM Orders O
JOIN orderDetails OD 
ON O.OrderID = OD.OrderID
JOIN Products P 
ON P.ProductID = OD.ProductID
JOIN Categories C 
ON C.CategoryID = P.CategoryID
)

SELECT
    strftime('%Y', OrderDate) AS Year,
    strftime('%m', OrderDate) AS Month,
    ROUND(SUM(Sales),2) AS TotalSales
    
FROM Salestotals_overtime
GROUP BY 1,2
ORDER BY 1,3;

      

 * sqlite:///Northwind.db
Done.


Year,Month,TotalSales
1996,08,26609.4
1996,09,27636.0
1996,07,30192.1
1996,10,41203.6
1996,11,49704.0
1996,12,50953.4
1997,06,39088.0
1997,03,39979.9
1997,02,41207.2
1997,11,45913.36


**strftime** is a function commonly used in SQL to format date and time values. The name "strftime" stands for "string format time," and it is often used to convert date and time values into a specified format. Uses:

-- Format the current date and time as 'YYYY-MM-DD HH:MM:SS'
SELECT strftime('%Y-%m-%d %H:%M:%S', 'now');

-- Extract the year from a date
SELECT strftime('%Y', '2023-11-21');

-- Extract the month from a date
SELECT strftime('%m', '2023-11-21');

-- Extract the day from a date
SELECT strftime('%d', '2023-11-21');


### VIEWS

A **view** is a virtual table that is based on the result of a SELECT query. Views allow you to encapsulate complex queries and provide a way to present the result of a query as if it were a table. They offer several benefits, including simplifying complex queries, enhancing security by restricting access to certain columns, and providing a consistent interface to users.

#### Design a view that shows customers who have not placed any orders.

In [17]:
%%sql

CREATE VIEW Customer_order AS 
SELECT 
      CompanyName,
      OrderID,
      OrderDate
    
FROM Customers C
LEFT JOIN Orders O
ON C.CustomerID = O.CustomerID

 * sqlite:///Northwind.db
(sqlite3.OperationalError) view Customer_order already exists
[SQL: CREATE VIEW Customer_order AS 
SELECT 
      CompanyName,
      OrderID,
      OrderDate
    
FROM Customers C
LEFT JOIN Orders O
ON C.CustomerID = O.CustomerID]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [18]:
%%sql

SELECT 
      DISTINCT CompanyName
FROM Customer_order
WHERE OrderID  ISNULL;

 * sqlite:///Northwind.db
Done.


CompanyName
FISSA Fabrica Inter. Salchichas S.A.
Paris spcialits
IT


#### Develop a view that combines information from multiple tables to display employee details along with their territories.

In [19]:
%%sql

CREATE VIEW Employee_full_details AS
SELECT
      E.LastName||' '||E.FirstName as Name,
      E.Title,
      E.TitleOfCourtesy,
      E.BirthDate,
      E.HireDate,
      E.HomePhone,
      E.City,
      E.Region,
      E.Country,
      T.TerritoryDescription,
      R.RegionDescription  
FROM Employees E
JOIN EmployeeTerritories ET
ON E.EmployeeID = ET.EmployeeID
JOIN Territories T
ON T.TerritoryID = ET.TerritoryID
JOIN Region R
ON R.RegionID = T.RegionID

 * sqlite:///Northwind.db
(sqlite3.OperationalError) view Employee_full_details already exists
[SQL: CREATE VIEW Employee_full_details AS
SELECT
      E.LastName||' '||E.FirstName as Name,
      E.Title,
      E.TitleOfCourtesy,
      E.BirthDate,
      E.HireDate,
      E.HomePhone,
      E.City,
      E.Region,
      E.Country,
      T.TerritoryDescription,
      R.RegionDescription  
FROM Employees E
JOIN EmployeeTerritories ET
ON E.EmployeeID = ET.EmployeeID
JOIN Territories T
ON T.TerritoryID = ET.TerritoryID
JOIN Region R
ON R.RegionID = T.RegionID]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Using the above VIEW to query the number of territories the employee serve

In [20]:
%%sql

SELECT
      Name as Employee,
      COUNT(TerritoryDescription) as Territories_served

FROM Employee_full_details
GROUP BY 1
ORDER BY 2 DESC;        

 * sqlite:///Northwind.db
Done.


Employee,Territories_served
King Robert,10
Fuller Andrew,7
Dodsworth Anne,7
Buchanan Steven,7
Suyama Michael,5
Leverling Janet,4
Callahan Laura,4
Peacock Margaret,3
Davolio Nancy,2


### Window Functions

**Window functions** are a category of functions in SQL that perform calculations across a specified range of rows related to the current row within the result set. These functions are applied to a "window" of rows defined by an OVER clause. Window functions are especially useful for performing calculations that involve multiple rows in a result set.

#### Rank products within each category based on their unit prices.

In [21]:
%%sql

WITH products_category AS
(SELECT
      CategoryName,
      ProductName,
      UnitPrice
    
FROM Products P
JOIN Categories C
ON C.CategoryID = P.CategoryID)

SELECT
     CategoryName,
     ProductName,
     RANK() OVER(PARTITION BY CategoryName ORDER BY UnitPrice) as Rank_
    
FROM products_category;    

 * sqlite:///Northwind.db
Done.


CategoryName,ProductName,Rank_
Beverages,Guaran Fantstica,1
Beverages,Rhnbru Klosterbier,2
Beverages,Sasquatch Ale,3
Beverages,Laughing Lumberjack Lager,3
Beverages,Outback Lager,5
Beverages,Chai,6
Beverages,Steeleye Stout,6
Beverages,Chartreuse verte,6
Beverages,Lakkalikri,6
Beverages,Chang,10


#### Calculate the running total of sales for each month using a window function.

In [22]:
%%sql

WITH order_details AS
(SELECT
     O.OrderID, 
     O.OrderDate,
     CAST(strftime('%Y', OrderDate) AS INT) as Year,
     CAST(strftime('%m', OrderDate) AS INT) as Month,
     OD.UnitPrice, 
     OD.Quantity,
     (OD.UnitPrice * OD.Quantity) as Sales    
        
FROM Orders O
JOIN OrderDetails OD
ON O.OrderID = OD.OrderID) 

SELECT
      Year,
      Month,
      ROUND(Sales,2) as Monthly_sales,
      ROUND(SUM(Sales) OVER(ORDER BY Year, Month),2) as Running_total

FROM order_details;

 * sqlite:///Northwind.db
Done.


Year,Month,Monthly_sales,Running_total
1996,7,168.0,30192.1
1996,7,98.0,30192.1
1996,7,174.0,30192.1
1996,7,167.4,30192.1
1996,7,1696.0,30192.1
1996,7,77.0,30192.1
1996,7,1484.0,30192.1
1996,7,252.0,30192.1
1996,7,100.8,30192.1
1996,7,234.0,30192.1


### Control Frow : CASE

The CASE statement in SQL is a powerful and flexible way to perform conditional logic within a query. It allows you to create conditional expressions and define different actions based on those conditions.

#### Query to display a column indicating whether an order is "Shipped" or "Not Shipped" based on the ShippedDate column.

In [23]:
%%sql

SELECT
      OrderID,
      CASE WHEN ShippedDate IS NOT NULL THEN 'Shipped'
           ELSE 'Not Shipped' END AS Shipping_status
      
FROM orders
LIMIT 5;

 * sqlite:///Northwind.db
Done.


OrderID,Shipping_status
10248,Shipped
10249,Shipped
10250,Shipped
10251,Shipped
10252,Shipped


#### Categorize products into three price ranges: "Low," "Medium," and "High."

In [24]:
%%sql

SELECT
      CategoryName,
      ProductName,
      UnitPrice,
      CASE WHEN UnitPrice <= 20 THEN 'Low'
           WHEN UnitPrice <= 50 THEN  'Medium' 
           ELSE 'High' END AS Price_range
    
FROM Products P
JOIN Categories C
ON C.CategoryID = P.CategoryID
LIMIT 6;

 * sqlite:///Northwind.db
Done.


CategoryName,ProductName,UnitPrice,Price_range
Beverages,Chai,18,Low
Beverages,Chang,19,Low
Condiments,Aniseed Syrup,10,Low
Condiments,Chef Anton's Cajun Seasoning,22,Medium
Condiments,Chef Anton's Gumbo Mix,21.35,Medium
Condiments,Grandma's Boysenberry Spread,25,Medium


#### Calculate the average order amount for each customer, and use a CASE statement to categorize customers as "Low," "Medium," or "High" based on their average order amount.

In [25]:
%%sql

SELECT 
      C.CompanyName,
      ROUND(AVG(OD.UnitPrice * OD.Quantity),2) as Avg_order_amt,
      CASE WHEN ROUND(AVG(OD.UnitPrice * OD.Quantity),2) >= 900 THEN 'High'
           WHEN ROUND(AVG(OD.UnitPrice * OD.Quantity),2) >= 300 THEN 'Medium'
           ELSE 'Low' END AS Customers_category

FROM Customers C
JOIN Orders O
ON O.CustomerID = C.CustomerID
JOIN OrderDetails OD
ON O.OrderID = OD.OrderID
GROUP BY 1
LIMIT 7;

 * sqlite:///Northwind.db
Done.


CompanyName,Avg_order_amt,Customers_category
Alfreds Futterkiste,383.02,Medium
Ana Trujillo Emparedados y helados,140.3,Low
Antonio Moreno Taquera,442.08,Medium
Around the Horn,460.22,Medium
B's Beverages,276.81,Low
Berglunds snabbkp,518.62,Medium
Blauer See Delikatessen,231.41,Low


#### Categorize orders as "On Time," "Delayed," or "Not Shipped" based on the difference between OrderDate and ShippedDate.

In [26]:
%%sql

WITH Shipping_days AS
(SELECT 
    OrderID,
    julianday(RequiredDate) - julianday(OrderDate) AS duration_days,
    julianday(ShippedDate) - julianday(OrderDate) AS Time_to_ship,
    (julianday(RequiredDate) - julianday(OrderDate))-(julianday(ShippedDate) - julianday(OrderDate)) as ETA_days
FROM 
    orders)

SELECT
      OrderID,
      CASE WHEN ETA_days > 0 THEN 'On Time'
           WHEN ETA_days <= 0 THEN 'Delayed'
           ELSE 'Not Shipped' END AS shipping_status

FROM Shipping_days;            

 * sqlite:///Northwind.db
Done.


OrderID,shipping_status
10248,On Time
10249,On Time
10250,On Time
10251,On Time
10252,On Time
10253,On Time
10254,On Time
10255,On Time
10256,On Time
10257,On Time


### Bonus - Data Analysis

####  Top-performing employees based on the total sales amount.

In [27]:
%%sql

SELECT
      E.LastName||' '||E.FirstName as Name,
      ROUND(OD.UnitPrice * OD.Quantity, 2) as Total_sales
        
FROM Employees E
JOIN Orders O
ON E.EmployeeID = O.EmployeeID
JOIN OrderDetails OD
ON OD.OrderID = O.OrderID
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///Northwind.db
Done.


Name,Total_sales
Fuller Andrew,936.0
Davolio Nancy,760.0
Dodsworth Anne,304.0
King Robert,240.0
Callahan Laura,204.0
Buchanan Steven,168.0
Suyama Michael,167.4
Leverling Janet,100.8
Peacock Margaret,77.0


#### Products with declining sales over the last quarter

In [28]:
%%sql

WITH previous_quarter AS
(SELECT
       P.ProductID,
       P.ProductName,
       ROUND(SUM((OD.UnitPrice * OD.Quantity)),2) as Sales
    
FROM Orders O
JOIN orderDetails OD 
ON O.OrderID = OD.OrderID
JOIN Products P 
ON P.ProductID = OD.ProductID
JOIN Categories C 
ON C.CategoryID = P.CategoryID
WHERE
     strftime('%Y', OrderDate) = '1997'
    AND strftime('%m', OrderDate) BETWEEN '10' AND '12'
GROUP BY 1,2
),

Current_quarter_sales AS
(SELECT
       P.ProductID,
       P.ProductName,
       ROUND(SUM((OD.UnitPrice * OD.Quantity)),2) as Sales
    
FROM Orders O
JOIN orderDetails OD 
ON O.OrderID = OD.OrderID
JOIN Products P 
ON P.ProductID = OD.ProductID
JOIN Categories C 
ON C.CategoryID = P.CategoryID
WHERE
     strftime('%Y', OrderDate) = '1998'
    AND strftime('%m', OrderDate) BETWEEN '01' AND '03'
GROUP BY 1,2 
)

SELECT
      PQ.ProductID,
      PQ.ProductName,
      PQ.Sales as Previous_quarter_sales,
      CQ.Sales as Current_quarter_sales,
      ROUND((CQ.Sales - PQ.Sales),2) as Difference
    
FROM previous_quarter PQ
LEFT JOIN Current_quarter_sales CQ
ON PQ.ProductID = CQ.ProductID
WHERE (CQ.Sales - PQ.Sales) < 0


 * sqlite:///Northwind.db
Done.


ProductID,ProductName,Previous_quarter_sales,Current_quarter_sales,Difference
2,Chang,2622.0,2527.0,-95.0
9,Mishi Kobe Niku,582.0,291.0,-291.0
17,Alice Mutton,7137.0,4992.0,-2145.0
19,Teatime Chocolate Biscuits,901.6,782.0,-119.6
28,Rssle Sauerkraut,4970.4,2872.8,-2097.6
34,Sasquatch Ale,924.0,420.0,-504.0
41,Jack's New England Clam Chowder,1688.75,1360.65,-328.1
43,Ipoh Coffee,4324.0,4140.0,-184.0
44,Gula Malacca,1945.0,778.0,-1167.0
45,Rogede sild,2099.5,769.5,-1330.0
